## Imports

In [ ]:
import os
import sys
import time
import logging
import importlib
import datetime
import re
import json
import glob
import gzip
from pathlib import Path
from IPython.display import display
from playsound import playsound
import numpy as np
from numpy.lib.recfunctions import join_by
from numpy.core.records import fromarrays
import matplotlib.pyplot as plt
from matplotlib.pyplot import pcolormesh
import pandas as pd
import aiohttp
import asyncpg
import requests
import math
import itertools
from functools import lru_cache
#@lru_cache(maxsize=None)

from edcompanion import pgsqldata, timetools, navroute, events
from edcompanion.timetools import make_datetime, make_naive_utc
from edcompanion.edsm_api import get_edsm_info, distance_between_systems
from edcompanion.pgsqldata import PGSQLDataSourceEDDB

def prettyprint(item):
    print(json.dumps(item, indent=4, sort_keys=False))
    
def record_to_dict(record):
    if isinstance(record,asyncpg.Record):
        return {k:v for k,v in record.items()}
    return {}

plt.rcParams["figure.figsize"] = (16,12)

pd.options.display.max_colwidth = 150


# Some vars 
logpath = "/Users/fenke/Saved Games/Frontier Developments/Elite Dangerous"
system_name = None

In [ ]:
  
planet_values = {
    False: { # was-not-discovered
        False: {# was-not-mapped
            False: { # is-not-terraformable
                'Water world': 1559138,
                'Earthlike body': 4224870,
                'Ammonia world': 2242455,
            },
            True : { # is-terraformable
                'Water world': 4198704,
                "High metal content body": 2562654,
                "Rocky body": 2024270
            }
        },
        
        True: {# was-mapped, this specic combo is rubbish
        }
    },
    True: { # was-discovered
        False: {# was-not-mapped
            False: { # is-not-terraformable
                'Water world': 1312209,
                'Earthlike body': 3555753,
                'Ammonia world': 1887305,
            },
            True : { # is-terraformable
                'Water world': 3533732,
                "High metal content body": 2156792,
                "Rocky body": 1703675
            }
        },
        
        True: {# was-mapped
            False: { # is-not-terraformable
                'Water world': 540297,
                'Earthlike body': 1464068,
                'Ammonia world': 777091,
            },
            True : { # is-terraformable
                'Water world': 1455001,
                "High metal content body": 888051,
                "Rocky body": 701482
            }
        }
    }
}



### Missions

In [ ]:
from itertools import permutations

def parse_route_systems(system_name, mission_db):
    return { **{
        system_name:np.asarray([get_edsm_info(system_name).get('coords',{}).get(k) for k in ['x', 'y', 'z']])
    }, **{
        s.get('DestinationSystem'):np.asarray([s.get('coords',{}).get(k) for k in ['x', 'y', 'z']])
        for s in mission_db.values()
    }}
 

def calculate_total_jumps(*route_points, jumpdistance=25):
    return sum([1+math.floor(distance_between_systems(*rp)/jumpdistance) for rp in route_points])

def get_mission_routes(start_system, mission_db, jumpdistance):
    route_systems = parse_route_systems(start_system, mission_db)
    all_routes = [r for r in permutations(route_systems) if r[0] == system_name]
    return sorted([[(x,y) for x,y in zip(r, r[1:])] for r in all_routes], key=lambda R:calculate_total_jumps(*R, jumpdistance=jumpdistance))


### PostgreSQL

In [ ]:
importlib.reload(pgsqldata)

eddatasource = pgsqldata.PGSQLDataSourceEDDB(os.getenv("PGSQL_URL"), server_settings={'search_path': "edsm"})

async def find_system(system, distance=40):
    pool = await eddatasource.pool.pgsql_pool()
    if isinstance(system, str):
        q1 = await pool.fetchrow(
            """
                SELECT s.*, 0 as distance, p.security
                FROM systems s
                LEFT JOIN populated p
                ON s.name = p.systemname
                where s.name = $1
            """, system)
        if not q1:
            return get_edsm_info(system)
        return q1
    
    assert len(system) == 3
    coordinates = system
    c20_location = [int(20*math.floor(v/20)) for v in coordinates]
    side = int(20*math.floor(distance/20))
    q1 = await pool.fetch(
        "SELECT systems.*, |/((x-$7)^2 + (y-$8)^2 + (z-$9)^2) as distance, populated.security "+
        "FROM systems "+
        "LEFT JOIN populated " +
        "ON systems.name = populated.systemname "
        "WHERE x>=$1 AND x<$2 AND  y>=$3 AND y<$4  AND  z>=$5 AND z<$6  AND |/((x-$7)^2 + (y-$8)^2 + (z-$9)^2) < $10"+
        "ORDER BY distance",
        *[d for c in coordinates for d in [c-40, c+40]], *coordinates, distance)
    if not q1:
        return q1
    return await find_system(q1[0].get("name"))

async def find_nearby_systems(system, distance, limit=5):
    if isinstance(system, str):
        ql = await find_system(system)
        coordinates = [ql.get(k) for k in ["x", "y","z"]]
    else:
        coordinates = system
        
    #c20_location = [int(20*math.floor(v/20)) for v in coordinates]
    side = int(20*math.ceil(distance/20))
    pool = await eddatasource.pool.pgsql_pool()

    return await pool.fetch(
        "SELECT name, x,y,z, |/((x-$7)^2 + (y-$8)^2 + (z-$9)^2) as distance "+
        "FROM systems "+"""
            WHERE  x>=$1 AND x<$2 
              AND  y>=$3 AND y<$4  
              AND  z>=$5 AND z<$6 
        """ +
        "  AND |/((x-$7)^2 + (y-$8)^2 + (z-$9)^2) < $10"+
        "ORDER BY distance LIMIT " + str(limit),
        *[d for c in coordinates for d in [c-side, c+side]], 
        *coordinates, distance)

In [ ]:
record = await find_system('Ix')

In [ ]:
record

In [ ]:
df = await eddatasource.get_dataframe(
            """
                SELECT s.*, 0 as distance, p.security
                FROM systems s
                LEFT JOIN populated p
                ON s.name = p.systemname
                where s.name like $1
            """, 'Ix%')

In [ ]:
df

In [ ]:
{k:v for k,v in record.items()}

In [ ]:
apiparams = dict(
    base_url="http://127.0.0.1:8000",
    headers=dict(
        authorization='9VknM2Gq2UcyM9tXJm43k7M4zVDNhVcQ' #os.getenv("USR_TOKEN")
    )
)
print(apiparams)

In [ ]:
async with aiohttp.ClientSession(**apiparams) as session:
    t0 = time.perf_counter_ns()
    req = await session.get("/systems", params={"name":"Ix"})
    t1 = time.perf_counter_ns()

    data = pd.read_json((await req.read()).decode("utf-8"), orient='split')
    print(f"Query took {(t1-t0)/1e6} ms for {data.shape[0]} rows")

    #plot(np.array(lttbdata[:,0], dtype='datetime64[s]'),lttbdata[:,1])

In [ ]:
async with aiohttp.ClientSession(**apiparams) as session:
    t0 = time.perf_counter_ns()
    req = await session.get("/systems", params={"coordinates":[-65,8,-111], "distance":12})
    t1 = time.perf_counter_ns()

    data = pd.read_json((await req.read()).decode("utf-8"), orient='split')
    print(f"Query took {(t1-t0)/1e6} ms for {data.shape[0]} rows")

    #plot(np.array(lttbdata[:,0], dtype='datetime64[s]'),lttbdata[:,1])

In [ ]:
data

In [ ]:
req

In [ ]:
(await req.read()).decode("utf-8")

In [ ]:
[k  for k in record.keys()]

In [ ]:
importlib.reload(logging)
logging.basicConfig(
    format="%(asctime)s.%(msecs)03d \t%(threadName)s\t%(name)s\t%(lineno)d\t%(levelname)s\t%(message)s",
    datefmt="%Y-%m-%dT%H:%M:%S",
    level=logging.INFO
)

logging.captureWarnings(True)



# Companion

## Lib Imports

In [ ]:
import edcompanion.eddbreader

## Galaxy

In [ ]:
importlib.reload(edcompanion.eddbreader)

#eddbfile = 'e:\data\eddb\systemsWithCoordinates7days.json.gz'

#eddbfile = 'e:\data\eddb\galaxy_stations.json.gz'
#eddbfile = 'e:\data\eddb\galaxy_populated.json.gz'
eddbfile = 'e:\data\eddb\galaxy_1day.json.gz'
keys = {}
count = 2
item = None
#for item in eddbreader.edc_dbfilereader(eddbfile, True, lambda X:item.get('name') == "Cygni X-3"):
for item in edcompanion.eddbreader.edc_dbfilereader(eddbfile, True):
    if not [k for k in item.get('bodies',[]) if 'ary' in k.get('type') ]:
        continue
    #if item.get('bodyCount',0) > 5:
    #    continue
    #prettyprint([k for k in item])
    prettyprint(item)

    count -= 1
    if not count > 0:
        break


In [ ]:
from edcompanion import eddbreader
importlib.reload(eddbreader)

#eddbfile = 'e:\data\eddb\systemsWithCoordinates7days.json.gz'

#eddbfile = 'e:\data\eddb\galaxy_stations.json.gz'
#eddbfile = 'e:\data\eddb\galaxy_populated.json.gz'
eddbfile = 'e:\data\eddb\galaxy_1day.json.gz'
keys = {}
count = 2
item = None
#for item in eddbreader.edc_dbfilereader(eddbfile, True, lambda X:item.get('name') == "Cygni X-3"):
for item in eddbreader.edc_dbfilereader(eddbfile, True):
    for key in item:
        if key not in keys:
            keys[key] = str(type(item[key]))
    continue
    if item.get('bodyCount',0) > 2:
        continue
    prettyprint([k for k in item])
    #prettyprint(item)

    count -= 1
    if not count > 0:
        break


In [ ]:
keys

In [ ]:
importlib.reload(eddbreader)
#eddbfile = 'e:\data\eddb\systemsWithCoordinates7days.json.gz'

#eddbfile = 'e:\data\eddb\galaxy_stations.json.gz'
#eddbfile = 'e:\data\eddb\galaxy_populated.json.gz'
eddbfile = 'e:\data\eddb\galaxy_1day.json.gz'
count = 5
item = None
key = 'stations'
systems_update = []
#for item in eddbreader.edc_dbfilereader(eddbfile, True, lambda X:item.get('name') == "Cygni X-3"):
for item in eddbreader.edc_dbfilereader(eddbfile, True):
    if not item.get(key) or not item.get('allegiance'):
        continue
    if len(item.get(key)) >4:
        continue
    if item.get('allegiance') == 'Independent':
        continue
    system_id = item.get('id64')
    
    systems_update.append(
        [item.get(k) for k in ['id64','name','coords', 'bodyCount']] + 
        [c for c in item.get('coords')]+
        [item.get('bodyCount')]
    )
    system_info = {k.lower():item.get(k) for k in ['id64','name','coords', 'bodyCount']}
    if key == 'stations':
        #if not [S.get('id') for S in item.get(key) if S.get('controllingFactionState') ]:
        #    continue
        system_info.update({k:item.get(k) for k in ['name','allegiance','government', 'primaryEconomy']})
        prettyprint(system_info)
        for station in item.get(key):
            print([k for k in station])
            prettyprint({
                k:station.get(k) 
                for k in [
                    'id', 'name', 'distanceToArrival', 
                    'type', 'government', 'primaryEconomy',
                    'controllingFaction', 'allegiance'
                ]})
    elif key == 'factions':
        prettyprint((system_info,item.get('allegiance'), item.get(key)))
    elif key == 'controllingFaction':
        if (
            item.get(key).get('government') != item.get('government')) or (
            item.get(key).get('allegiance') != item.get('allegiance')):
            
            prettyprint({k:item.get(k) for k in ['name','allegiance','government', 'primaryEconomy', 'controllingFaction']})
        else:
            continue
    else:
        prettyprint(system_info.update({k:item.get(k) for k in ['name','allegiance','government', 'primaryEconomy']+[key]}))
        
        
    
    count -= 1
    if not count > 0:
        break


In [ ]:
item.get(key)

In [ ]:
importlib.reload(eddbreader)

eddbfile = 'e:\data\eddb\systemsWithCoordinates7days.json.gz'

#eddbfile = 'e:\data\eddb\galaxy_stations.json.gz'
#eddbfile = 'e:\data\eddb\galaxy_populated.json.gz'
#eddbfile = 'e:\data\eddb\galaxy_1days.json.gz'
count = 2
#for item in eddbreader.edc_dbfilereader(eddbfile, True, lambda X:item.get('name') == "Cygni X-3"):
for item in eddbreader.edc_dbfilereader(eddbfile, True):
    #prettyprint([k for k in item])
    continue
    prettyprint(item)
    break
    count -= 1
    if not count > 0:
        break


## Powerplay

In [ ]:
importlib.reload(eddbreader)


eddbfile = '../scripts/data/powerPlay.json.gz'
count = 3
for item in eddbreader.edc_dbfilereader(eddbfile, True):
    prettyprint(item)
    if not count > 0:
        break
    count -= 1

## Time needed

In [ ]:
importlib.reload(eddbreader)

In [ ]:
eddbfile = 'e:\data\eddb\galaxy_1day.json.gz'
def process_data(data):
    pass
    
eddbreader.edc_dbfile_process(eddbfile, lambda D: process_data(D), True)


## material

In [ ]:
# cool explorer
materials_needed = """

Conductive Ceramics	Mfc	G3	7
Conductive Components	Mfc	G2	1
Decoded Emission Data	Enc	G4	4
Germanium	Raw	G2	5
Grid Resistors	Mfc	G1	5
Heat Exchangers	Mfc	G3	3
Heat Vanes	Mfc	G4	6
Hybrid Capacitors	Mfc	G2	3
Iron	Raw	G1	10
Irregular Emission Data	Enc	G2	4
Manganese	Raw	G2	5
Modified Consumer Firmware	Enc	G2	4
Phosphorus	Raw	G1	1
Salvaged Alloys	Mfc	G1	5
Security Firmware Patch	Enc	G4	1
Specialised Legacy Firmware	Enc	G1	1
Unexpected Emission Data	Enc	G3	6
Vanadium	Raw	G2	3


"""

In [ ]:
# cool rescue
materials_needed = """

Grid Resistors	Mfc	G1	5
Heat Exchangers	Mfc	G3	3
Heat Vanes	Mfc	G4	1
Iron	Raw	G1	6
Irregular Emission Data	Enc	G2	5
Vanadium	Raw	G2	3

"""

In [ ]:
matmap = dict(Mfc='Manufactured', Raw='Raw', Enc='Encoded')
required_mats = [
    [
        f(c) for c,f in zip(l.split('\t'),(
            lambda S:S,
            lambda S:matmap.get(S),
            lambda S:int(S[1]),
            lambda S:int(S)
        )) 
    ]
    for l in materials_needed.splitlines() if l
    if l
]
pd.DataFrame(required_mats, columns=['material', 'type', 'grade','qty']).set_index(['type','material']).sort_values(['type','material'])

In [ ]:
pd.DataFrame([
    R+
    [my_materials.get(R[1],{}).get(R[0].lower().replace(' ',''),0)] 
    for R in required_mats
], columns=['Material','Type','Grade','Needed','Available']).set_index(['Type','Material']).sort_values(['Type','Material'])

In [ ]:
my_materials

Nuith's Grace
Gnade der Nacht

Nut, also Nuit or Nuith are names for the Egyptian godess of the sky and the stars, cosmos, mothers, astronomy and the universe.


## Navigation route

In [ ]:
from edcompanion import navroute
from edcompanion import events

In [ ]:
logging.getLogger(f"root").setLevel(logging.INFO)

In [ ]:
importlib.reload(navroute)
logging.getLogger(f"root.{navroute}").setLevel(logging.INFO)
importlib.reload(events)
logging.getLogger(f"root.{events}").setLevel(logging.INFO)

In [ ]:
item = {}
for item in navroute.edc_navigationroute(logpath):

    sys.stdout.write(f"\r{item.get('StarPos')[0]:8,.0F} {item.get('StarPos')[1]:8,.0F} {item.get('StarPos')[1]:8,.0F} | ")
    sys.stdout.write(f" {item.get('StarClass'):4} | {item.get('StarSystem'):32} | {'scoopable' if item.get('StarClass') in 'KGBFOAM' else ''}\n")

print(item)

In [ ]:
pos = np.asarray([389.25, 731.4375, 662.78125])
df = pd.DataFrame([
    (
        item.get('StarSystem'),
        np.asarray(item.get('StarPos')),
        item.get('StarClass'),
        np.sqrt(np.sum(np.square(np.asarray(item.get('StarPos'))-pos)))
        

    ) for item in navroute.edc_navigationroute(logpath)
    if item.get('StarClass') in 'KGBFOAM'
], columns=['System','coord','class', 'distance']).sort_values(['distance'])

In [ ]:
type(df.iloc(0,0)

In [ ]:
str(df['System'].iloc[0])


In [ ]:
system_name

In [ ]:
navi_route = {item.get('StarSystem'):item for item in navroute.edc_navigationroute(logpath)}
navi_distances = {s:np.sqrt(np.sum(np.square(np.asarray(i.get('StarPos'))-starpos))) for s, i in navi_route.items() if s != system_name}


In [ ]:
navi_distances = {s:i for s,i in navi_distances.items() if i > 0}

In [ ]:
list(navi_distances)[0]

In [ ]:
from itertools import accumulate
from functools import reduce

In [ ]:
list(navi_distances)

In [ ]:
def accufunc(total, item):
    return item if navi_distances[item] < navi_distances[total] else total

result = [x for x in accumulate(list(navi_distances), accufunc, initial=list(navi_distances)[0])]

In [ ]:
result[0]

In [ ]:
reduce(accufunc, list(navi_distances))

In [ ]:
print([x for x in accumulate(navi_distances, accufunc, initial=list(navi_distances)[0])][0])

In [ ]:
navi_distances

In [ ]:
[item.get('StarSystem') for item in navroute.edc_navigationroute(logpath)]

### Loadout & Modules

In [ ]:
json.loads('[{"header":{"appName":"EDSY","appVersion":38445,"appURL":"https://edsy.org/#/L=Gr00000H4C0S40,,,9p310A5UH054_W0AN8G032mO0AcII05J_W40upD6qpD8qpDGyPcAsOG02m_W0B2uI03KxS6IkPcKx8JQkPcSx8JYkPcax8JBK4I02mjB32xLQ6upDIqpDBb610,,34a300M2904_w307Q4I02nyD52vR26qpDAupDIj7iKni9mpT03w000Iw902jwG08Wgu1OrPJ6e490"},"data":{"event":"Loadout","Ship":"dolphin","ShipName":"","ShipIdent":"","HullValue":1095780,"ModulesValue":18017610,"UnladenMass":166.405426,"CargoCapacity":4,"MaxJumpRange":65.603123,"FuelCapacity":{"Main":16,"Reserve":0.5},"Rebuy":955669,"Modules":[{"Slot":"CargoHatch","Item":"modularcargobaydoor","On":true,"Priority":4},{"Slot":"Armour","Item":"dolphin_armour_grade1","On":true,"Priority":1,"Value":0},{"Slot":"PowerPlant","Item":"int_powerplant_size3_class5","On":true,"Priority":1,"Value":480410,"Engineering":{"BlueprintName":"PowerPlant_Stealth","Level":4,"Quality":1,"ExperimentalEffect":"special_powerplant_cooled","Modifiers":[{"Label":"Mass","Value":2.9,"OriginalValue":2.5},{"Label":"PowerCapacity","Value":10.56,"OriginalValue":12},{"Label":"HeatEfficiency","Value":0.162,"OriginalValue":0.4}]}},{"Slot":"MainEngines","Item":"int_engine_size4_class2","On":true,"Priority":0,"Value":59630,"Engineering":{"BlueprintName":"Engine_Tuned","Level":4,"Quality":0.774,"ExperimentalEffect":"special_engine_overloaded","Modifiers":[{"Label":"Integrity","Value":56.32,"OriginalValue":64},{"Label":"PowerDraw","Value":4.1328,"OriginalValue":3.69},{"Label":"EngineOptimalMass","Value":289.8,"OriginalValue":315},{"Label":"EngineOptPerformance","Value":126.7448,"OriginalValue":100},{"Label":"EngineHeatRate","Value":0.747318,"OriginalValue":1.3}]}},{"Slot":"FrameShiftDrive","Item":"int_hyperdrive_size4_class5","On":true,"Priority":2,"Value":1610080,"Engineering":{"BlueprintName":"FSD_LongRange","Level":5,"Quality":1,"ExperimentalEffect":"special_fsd_heavy","Modifiers":[{"Label":"Mass","Value":13.000031,"OriginalValue":10},{"Label":"Integrity","Value":78.19986,"OriginalValue":100},{"Label":"PowerDraw","Value":0.517501,"OriginalValue":0.45},{"Label":"FSDOptimalMass","Value":846.293335,"OriginalValue":525}]}},{"Slot":"LifeSupport","Item":"int_lifesupport_size4_class2","On":true,"Priority":0,"Value":28370,"Engineering":{"BlueprintName":"Misc_LightWeight","Level":3,"Quality":1,"Modifiers":[{"Label":"Mass","Value":1.4,"OriginalValue":4},{"Label":"Integrity","Value":50.4,"OriginalValue":72}]}},{"Slot":"PowerDistributor","Item":"int_powerdistributor_size2_class2","On":true,"Priority":2,"Value":3620,"Engineering":{"BlueprintName":"PowerDistributor_HighFrequency","Level":5,"Quality":0.951,"ExperimentalEffect":"special_powerdistributor_lightweight","Modifiers":[{"Label":"Mass","Value":0.9,"OriginalValue":1},{"Label":"WeaponsCapacity","Value":13.300011,"OriginalValue":14},{"Label":"WeaponsRecharge","Value":2.312964,"OriginalValue":1.6},{"Label":"EnginesCapacity","Value":10.450008,"OriginalValue":11},{"Label":"EnginesRecharge","Value":0.867361,"OriginalValue":0.6},{"Label":"SystemsCapacity","Value":10.450008,"OriginalValue":11},{"Label":"SystemsRecharge","Value":0.867361,"OriginalValue":0.6}]}},{"Slot":"Radar","Item":"int_sensors_size3_class2","On":true,"Priority":2,"Value":10130,"Engineering":{"BlueprintName":"Sensor_LightWeight","Level":3,"Quality":0.72275,"Modifiers":[{"Label":"Mass","Value":1.083191,"OriginalValue":2},{"Label":"Integrity","Value":35.699844,"OriginalValue":51},{"Label":"SensorTargetScanAngle","Value":25.499954,"OriginalValue":30}]}},{"Slot":"FuelTank","Item":"int_fueltank_size4_class3","On":true,"Priority":1,"Value":24730},{"Slot":"Slot01_Size5","Item":"int_guardianfsdbooster_size5","On":true,"Priority":3,"Value":6483100},{"Slot":"Slot02_Size4","Item":"int_repairer_size4_class5","On":false,"Priority":1,"Value":4723920},{"Slot":"Slot03_Size4","Item":"int_fuelscoop_size4_class5","On":true,"Priority":3,"Value":2862360},{"Slot":"Slot04_Size3","Item":"int_shieldgenerator_size3_class2","On":true,"Priority":2,"Value":18810,"Engineering":{"BlueprintName":"ShieldGenerator_Optimised","Level":3,"Quality":0.96325,"ExperimentalEffect":"special_shield_toughened","Modifiers":[{"Label":"Mass","Value":1.322205,"OriginalValue":2},{"Label":"Integrity","Value":37.144933,"OriginalValue":38},{"Label":"PowerDraw","Value":1.007996,"OriginalValue":1.44},{"Label":"ShieldGenOptimalMass","Value":158.399849,"OriginalValue":165},{"Label":"ShieldGenStrength","Value":98.001137,"OriginalValue":90}]}},{"Slot":"Slot06_Size2","Item":"int_cargorack_size2_class1","On":true,"Priority":0,"Value":3250},{"Slot":"Slot07_Size2","Item":"int_repairer_size2_class5","On":false,"Priority":1,"Value":1458000},{"Slot":"Slot08_Size1","Item":"int_detailedsurfacescanner_tiny","On":true,"Priority":0,"Value":250000,"Engineering":{"BlueprintName":"Sensor_Expanded","Level":2,"Quality":0.686,"Modifiers":[{"Label":"DSS_PatchRadius","Value":23.37204,"OriginalValue":20}]}},{"Slot":"Slot09_Size1","Item":"int_dronecontrol_repair_size1_class2","On":false,"Priority":1,"Value":1200}]}}]')


In [ ]:
[k for k in current_ship]

## Live tracking the journal

In [ ]:
rankings = dict(
    Progress = dict(),
    Rank = dict()
    
)
modules = []
ships = {}
current_ship = None
my_materials = {}
missions = {}
completed = {}
signals = {}

In [ ]:
import edcompanion.navroute
import edcompanion.events
importlib.reload(edcompanion.events)
from edcompanion.events import edc_track_journal
jumptimes = []
system_name = ''
system_factions = []
entrytime = 0
body_id = 0
jumpdistance = 10
mission_advice = ""

for event in edc_track_journal(logpath, backlog=3):
    timestamp = make_datetime(event.pop("timestamp"))
    eventname = event.pop("event")
    
    if not jumptimes:
        entrytime=timestamp.timestamp()
        jumptimes.append(timestamp.timestamp())
    if not system_name and eventname == 'Location':
        system_name = event.get('StarSystem')
        
    sys.stdout.write(
        f"\r{str(timestamp)[:-6]:20} "+
        f"{timestamp.timestamp()-jumptimes[-1]:9,.0F} | {eventname:18} | {system_name:26} | ")
    if eventname == 'Journal':
        sys.stdout.write(f"{event.get('filename')}\n")
    elif eventname == 'Fileheader':
        sys.stdout.write(f"{'Odyssey' if event.get('Odyssey', False) else 'Horizons' }\n")

    elif eventname == 'FSDJump':
        entrytime=timestamp.timestamp()
        body_id = event.get('BodyID')
        system_name = event.get('StarSystem','')
        jumptimes.append(entrytime)
        if mission_advice and not [True for item in navroute.edc_navigationroute(logpath) if item.get('StarSystem') == mission_advice]:
            mnames = ', '.join([m.get('LocalisedName', '') for i,m in missions.items() if mission_advice==m.get('DestinationSystem') ])
            sys.stdout.write(f"Travel to {mission_advice} for \"{mnames}\"\n")
            continue
        system_factions = [f.get('Name','') for f in sorted(event.get('Factions',[{}]),key=lambda X: -X.get('Influence',0))]
        if len(system_factions) > 1:
            sys.stdout.write(f"Faction: {system_factions[0]}\n")
        
        
        sys.stdout.write(f"\n")
        #sys.stdout.write(f"{system_name:22}\n")
        continue
        
    elif eventname == 'LoadGame':
        entrytime=timestamp.timestamp()
        jumptimes.append(entrytime)
        #sys.stdout.write(f"\n")
        
    elif eventname == 'StartJump' and event.get("JumpType") == "Hyperspace":
        system_name = event.get('StarSystem','')
        sys.stdout.write(f"{system_name:22}\n")
        continue
        
    elif eventname == 'Scan' and event.get('BodyID') == body_id:
        sys.stdout.write(f"Class {event.get('StarType')}{event.get('Subclass')}")
        if not event.get('WasDiscovered'):
            sys.stdout.write(f"\t{'Undiscovered'}\n")
            playsound('./sound88.wav')
        else:
            sys.stdout.write(f"\t{'Previously Discovered'}\n")
            
        continue
        
    elif 'FSSSignalDiscovered' == eventname:
        signal = event.get("SignalName_Localised", None)
        if signal:
            if signal not in signals:
                signals[signal] = set()
            signals.get(signal).add((system_name,system_factions[0] if system_factions else '-'))

    elif 'scan' in eventname.lower():
        if not event.get("WasDiscovered", True):
            if event.get('TerraformState') == 'Terraformable' or planet_values.get(
                event.get("WasDiscovered"),{}).get(
                event.get("WasMapped"),{}).get(
                bool(event.get('TerraformState') == 'Terraformable'),{}).get(
                event.get("PlanetClass"),0) > 0:
        
                sys.stdout.write(f"{event.get('BodyName').replace(system_name,'')} {event.get('PlanetClass')} {event.get('TerraformState')}\n")

        if False and "Resource" in json.dumps(event):
            print(eventname, event)
            break

   

        continue
            
    elif 'Interdict' in eventname:
        sys.stdout.write(f"{event.get('Interdictor'):22}\n")
        continue

    elif 'Mission' in eventname:
        if eventname == 'Missions':
            active_missions = set([M.get('MissionID', 0) for M in event.get('Active',[])])
            for mid in active_missions:
                if mid not in missions:
                    missions[mid] = {}
            
            for mid in set(missions):
                if mid not in active_missions:
                    missions.get(mid).update({
                        eventname: timestamp.isoformat()
                    })
                    completed[mid] = missions.pop(mid)
            if not missions:
                mission_advice = ''
        else:   
            if eventname == 'MissionAbandoned' or  eventname == 'MissionCompleted' :
                completed[event.get('MissionID')] = missions.pop(event.get('MissionID'))
                if not missions:
                    mission_advice = ''

            elif eventname == 'MissionAccepted': 
                missions[event['MissionID']]={
                    k:event.get(k, '').split('$')[0] for k in ['LocalisedName', 'Expiry', 'DestinationSystem', 'DestinationStation']
                }
                #sys.stdout.write(f"({len(missions)}) {missions[event['MissionID']].get('LocalisedName')} -> {missions[event['MissionID']].get('DestinationSystem')}\n")

            elif eventname == 'MissionRedirected': 
                missions[event['MissionID']].update({
                    k:event.get('New'+k) for k in ['DestinationSystem', 'DestinationStation']
                })
                #sys.stdout.write(f"({len(missions)}) {missions[event['MissionID']].get('LocalisedName')} -> {missions[event['MissionID']].get('DestinationSystem')}\n")

            missions.get(event['MissionID'], {}).update({
                eventname: timestamp.isoformat(),
                'coords': get_edsm_info(missions.get(event['MissionID'], {}).get('DestinationSystem')).get('coords',[])
            })
        continue    
        ordered_routes = get_mission_routes(system_name, missions, jumpdistance=jumpdistance)
        if ordered_routes:
            best_route = ordered_routes[0]
            if best_route:
                s1, s2 = best_route[0]
                mission_advice = s2
                mnames = ', '.join([m.get('LocalisedName', '') for i,m in missions.items() if s2==m.get('DestinationSystem') ])
                sys.stdout.write(f"Travel to {s2} ({1+math.floor(distance_between_systems(s1, s2)/jumpdistance)}) for \"{mnames}\"\n")
                continue
        else:
            mission_advice = ''
        #sys.stdout.write(f"({len(missions)}) {missions.get(event['MissionID']).get('LocalisedName')} -> {missions.get(event['MissionID']).get('DestinationSystem')}\n")

    elif eventname == 'StoredModules' :
        #sys.stdout.write(f"{event.get('StationName',''):22}\n")
        modules = event.get('Items',[{}])
        continue
        
    elif ('Loadout' == eventname ):
        ships[event.get('ShipID')] = event.copy()
        current_ship = ships[event.get('ShipID')]
        jumpdistance = round(event.get('MaxJumpRange', 10),1)
        sys.stdout.write(f"{event.get('Ship',''):15} {event.get('ShipIdent',''):6} {event.get('MaxJumpRange',''):5,.1F} ly\n")
        continue
        
    elif (eventname == 'Rank' or eventname == 'Progress'):
        rankings[eventname].update(event)
        continue
        
    elif eventname == "Materials":
        my_materials = {T:{I.get("Name"):I.get("Count") for I in event.get(T,[])} for T in ['Raw','Encoded','Manufactured']}
        #my_materials = {I.get("Name_Localised", I.get("Name")):I.get("Count") for I in item.get('Raw',[]) + item.get("Encoded",[])}
        continue


#print(f"\nDone")
#print(maxlen)

In [ ]:
missions

### Signals

In [ ]:
pd.DataFrame([
    (n[0], n[1], s)
    for s, sn in signals.items()
    for n in sn
    #if 'Lavign' in n[1]
    #and 'Resource' in s
    #and 'Hazardous' not in s
], columns = ['system', 'faction', 'signal']).set_index(['faction','system']).sort_values('faction')

In [ ]:
missions

### Missions

In [ ]:
completed

In [ ]:
from itertools import permutations

def parse_route_systems(system_name, mission_db):
    return { **{
        system_name:np.asarray([get_edsm_info(system_name).get('coords',{}).get(k) for k in ['x', 'y', 'z']])
    }, **{
        s.get('DestinationSystem'):np.asarray([s.get('coords',{}).get(k) for k in ['x', 'y', 'z']])
        for s in mission_db.values()
    }}
 

def calculate_total_jumps(*route_points, jumpdistance=25):
    return sum([1+math.floor(distance_between_systems(*rp)/jumpdistance) for rp in route_points])

def get_mission_routes(start_system, mission_db, jumpdistance):
    route_systems = parse_route_systems(start_system, mission_db)
    all_routes = [r for r in permutations(route_systems) if r[0] == system_name]
    return sorted([[(x,y) for x,y in zip(r, r[1:])] for r in all_routes], key=lambda R:calculate_total_jumps(*R, jumpdistance=jumpdistance))


In [ ]:
get_mission_routes('Ix', completed, 35)

In [ ]:
for R in get_mission_routes('Ix', completed, 45):
    print(f"{calculate_total_jumps(*R, jumpdistance=45)} {R}")

In [ ]:
system_name = 'Ix'

In [ ]:
route_systems = parse_route_systems('Ix', completed)

In [ ]:
all_routes = [r for r in permutations(route_systems) if r[0] == system_name]

In [ ]:
all_routes

In [ ]:
for R in [[(x,y) for x,y in zip(r, r[1:])] for r in all_routes]:
    print(f"{calculate_total_jumps(*R, jumpdistance=12)} {R}")

In [ ]:
route_edges = sorted([[(x,y) for x,y in zip(r, r[1:])] for r in all_routes], key=lambda R:calculate_total_jumps(*R, jumpdistance=12))

In [ ]:
for R in get_mission_routes('Ix', completed, 35):
    print(f"{calculate_total_jumps(*R, jumpdistance=35)} {R}")

In [ ]:
print(route_edges)

In [ ]:
t = [('Ix', 'Xi Saon'), ('Xi Saon', 'Chias Vega'), ('Chias Vega', 'He Qiong')]
print([distance_between_systems(*rp) for rp in t])

In [ ]:
calculate_total_jumps(*t,jumpdistance=35)

0-0 0-1 0-2 0-3 0-4 
1-0 1-1 1-2 1-3 1-4
2-0 2-1 2-2 2-3 2-4
3-0 3-1 3-2 3-3 3-4
4-0 4-1 4-2 4-3 4-4

In [ ]:
def permute(alist):
    #print(f"List: {alist}")
    if len(alist) <= 1:
        yield alist
        alist = []

    for s in alist:
        rest = list(alist[:])
        rest.remove(s)
        for p in permute(rest):
            yield [s]+p


def test_permute(lst):
    print('\n%s permute(%s):' % ('-' * 20, lst))
    for p in permute(lst):
        print(p)




In [ ]:
test_permute([1,2,3,4])

In [ ]:
sorted((4,3))

In [ ]:
test_permute('abc')
test_permute('')
test_permute([1])

test_chosenk([1,2,3,4], 2)
test_chosenk('abc', 2)
test_chosenk('abc', 1)
test_chosenk('abc', 0)
test_chosenk('a', 2)
test_chosenk('', 2)

In [ ]:
4.2 * 1000 * 1000 * 50 / (1e3 * 3600)


In [ ]:
3600 

In [ ]:
from itertools import permutations, combinations

In [ ]:
[c for c in permutations(range(6),2)]

In [ ]:
[c for c in combinations(range(7),2)]

In [ ]:
set([v.get('DestinationSystem') for v in missions.values()])

### Ranks

In [ ]:
if rankings.get('Rank') and rankings.get('Progress'):
    rankinfo = [
        tuple([R] + [v.get(R,0) for r,v in rankings.items()])
        for R in rankings.get('Progress')]
    with pd.option_context('display.max_rows', len(rankinfo)+1, 'display.max_columns', len(rankinfo[0])+1):
        display(pd.DataFrame(
            rankinfo,
            columns=['rank', '% next', 'level']))

### Engineered Modules

In [ ]:
{M.get('Name_Localised').replace(' ','').lower():M.get('Name_Localised') for M in modules}

In [ ]:
prettyprint({
    M.get('Item').split('_')[1]: ''
    for I,S in ships.items() for M in S.get('Modules') if M.get('Engineering', False)
    if M.get('Item').split('_')[1] not in module_name_map
})

In [ ]:
module_name_map = {
    'engine':'Thrusters',
    'armour':'Armour',
    'powerdistributor':'Power Distributor',
    'detailedsurfacescanner': 'Surface Scanner',
    'multi-cannon': 'Multi-Cannon',
    'railgun': 'Rail Gun',
    'pulselaser': 'Pulse Laser',
    'ecpassengercabin': 'EC Passenger Cabin',
    'hullreinforcement': 'Hull Reinforcement',
    'modulereinforcement': 'Module Reinforcement',
    'powerplant': 'Power Plant',
    'afmunit': 'AFM Unit',
    'burstlaser': 'Burst Laser',
    'plasmaaccelerator':'Plasma Accelerator',
    'powerdistributor': 'Power Distributor',
    'sensors': 'Sensors',
    'cannon': 'Cannon',
    'plasmaacc': 'Plasma Acc',
    'guardianfsdbooster': 'Guardian FSD Booster',
    'cargorack': 'Cargo Rack',
    'bcpassengercabin': 'BC Passenger Cabin',
    'thrusters': 'Thrusters',
    'shieldgenerator': 'Shield Generator',
    'shieldbooster':'Shield Booster',
    'fcpassengercabin': 'FC Passenger Cabin',
    'hyperdrive': 'FSD',
    'lifesupport':'Life Support',
    'lcpassengercabin': 'LC Passenger Cabin',
    'crimescanner': 'Warrant Scanner', 
    'cloudscanner': 'Wake Scanner'
}

if modules:
    df_module_info = pd.DataFrame([
        (
            M.get('Name_Localised'),
            M.get('StarSystem'),
            '',
            M.get('Name').split('_')[-3],
            M.get('Name').split('_')[-2],
            M.get('EngineerModifications','._.').split('_')[-1],
            '',
            M.get('Level'),
            M.get('Quality')
        )
        for M in modules if M.get('EngineerModifications')] + [
        (
            module_name_map.get(M.get('Item').split('_')[1],M.get('Item').split('_')[1]),
            f"{S.get('ShipIdent')} / {S.get('Ship')}" ,
            M.get('Slot'),
            M.get('Item').split('_')[-2],
            M.get('Item').split('_')[-1],
            M.get('Engineering').get('BlueprintName').split('_')[-1],
            M.get('Engineering').get('ExperimentalEffect_Localised', ''),
            M.get('Engineering').get('Level'),
            M.get('Engineering').get('Quality'),

        ) for I,S in ships.items() for M in S.get('Modules') if M.get('Engineering', False)],
            columns=['Name', 'Location', 'Moduleslot', 'Class', 'Grade', 'Modification', 'Experimental', 'Level', 'Quality']
        ).set_index(['Name', 'Class', ])
    with pd.option_context('display.max_rows', len(df_module_info)+1, 'display.max_columns', len(df_module_info.columns)+1):
        display(df_module_info.sort_values(['Name','Moduleslot', 'Class', 'Grade']))
        
#print(df_modules).sort_values(['Name','Moduleslot', 'Class', 'Grade'])

In [ ]:
with pd.option_context('display.max_rows', len(df_module_info)+1, 'display.max_columns', len(df_module_info.columns)+1):
    display(df_module_info.reset_index().set_index(['Location','Name']).sort_values(['Location','Name','Moduleslot', 'Class', 'Grade']))


In [ ]:
df_module_info

Salutations Commanders

With the Thargoids on their way towards the bubble I was thinking about preparing
for what might be coming. As I don't expect to get a AX fighter ready in time what
I'm looking at station rescue with a cold running Dolphin, which would also come
in handy for avoiding detection by Thargoids. 

Interestingly the core internals for rescue and exploration are not that different.
My current exploration build is
The armoured 2A power plant would be replaced 

In [ ]:
len(df_module_info.columns)


In [ ]:
grid =[[int(R[0]),R[2]] for R in [ [f for f in L.split('\t')] for L in f"""
1	16	Charles Leclerc	FERRARI	1:21.280	1:21.208	1:20.161	14
2	1	Max Verstappen	RED BULL RACING RBPT	1:20.922	1:21.265	1:20.306	16
3	55	Carlos Sainz	FERRARI	1:21.348	1:20.878	1:20.429	13
4	11	Sergio Perez	RED BULL RACING RBPT	1:21.495	1:21.358	1:21.206	15
5	44	Lewis Hamilton	MERCEDES	1:22.048	1:21.708	1:21.524	17
6	63	George Russell	MERCEDES	1:21.785	1:21.747	1:21.542	17
7	4	Lando Norris	MCLAREN MERCEDES	1:22.130	1:21.831	1:21.584	19
8	3	Daniel Ricciardo	MCLAREN MERCEDES	1:22.139	1:21.855	1:21.925	20
9	10	Pierre Gasly	ALPHATAURI RBPT	1:22.010	1:22.062	1:22.648	18
10	14	Fernando Alonso	ALPINE RENAULT	1:22.089	1:21.861		17
11	31	Esteban Ocon	ALPINE RENAULT	1:22.166	1:22.130		12
12	77	Valtteri Bottas	ALFA ROMEO FERRARI	1:22.254	1:22.235		12
13	45	Nyck De Vries	WILLIAMS MERCEDES	1:22.567	1:22.471		13
14	24	Zhou Guanyu	ALFA ROMEO FERRARI	1:22.003	1:22.577		12
15	22	Yuki Tsunoda	ALPHATAURI RBPT	1:22.020			6
16	6	Nicholas Latifi	WILLIAMS MERCEDES	1:22.587			7
17	5	Sebastian Vettel	ASTON MARTIN ARAMCO MERCEDES	1:22.636			7
18	18	Lance Stroll	ASTON MARTIN ARAMCO MERCEDES	1:22.748			9
19	20	Kevin Magnussen	HAAS FERRARI	1:22.908			10
20	47	Mick Schumacher	HAAS FERRARI	1:23.005			9""".splitlines() if L]]

In [ ]:
grid.sort()

In [ ]:
grid

In [ ]:
grid[7][0] = 20

In [ ]:
grid

In [ ]:
grid.sort()

In [ ]:
grid = [[p,n] for p,n in zip(range(1,21), [g[1] for g in grid])]

## Ingest Journal

In [ ]:
prettyprint(journal_story)

In [ ]:
sorted(journal_events.keys())

In [ ]:
journal_story = {
    "current": {},
    "Commanders": {}
}
missions = {}

In [ ]:
journal_events = {
	#'Fileheader': lambda N, E: (),
	'Music': lambda N, E: (),
	'EngineerProgress': lambda N, E: (),
	#'LoadGame': lambda N, E: (),
	'Statistics': lambda N, E: (),
	'FSSSignalDiscovered': lambda N, E: (),
	'Location': lambda N, E: (),
	'ShipLocker': lambda N, E: (),
	#'Loadout': lambda N, E: (),
	'Cargo': lambda N, E: (),
	'NavRoute': lambda N, E: (),
	'FSDTarget': lambda N, E: (),
	'ModuleInfo': lambda N, E: (),
	'Undocked': lambda N, E: (),
	'Scanned': lambda N, E: (),
	'ReceiveText': lambda N, E: (),
	'StartJump': lambda N, E: (),
	'SupercruiseEntry': lambda N, E: (),
	'ApproachBody': lambda N, E: (),
	'ApproachSettlement': lambda N, E: (),
	'SupercruiseExit': lambda N, E: (),
	'DockingRequested': lambda N, E: (),
	'DockingGranted': lambda N, E: (),
	'Docked': lambda N, E: (),
	'RefuelAll': lambda N, E: (),
	'RedeemVoucher': lambda N, E: (),
	'LeaveBody': lambda N, E: (),
	'NewCommander': lambda N, E: (),
	'SendText': lambda N, E: (),
	'Market': lambda N, E: (),
	'FSDJump': lambda N, E: (),
	'DockingDenied': lambda N, E: (),
	'CargoDepot': lambda N, E: (),
	'USSDrop': lambda N, E: (),
	'MaterialDiscovered': lambda N, E: (),
	'MaterialCollected': lambda N, E: (),
	'ShipTargeted': lambda N, E: (),
	'ReservoirReplenished': lambda N, E: (),
	'NavBeaconScan': lambda N, E: (),
	'Scan': lambda N, E: (),
	'Promotion': lambda N, E: (),
	'Outfitting': lambda N, E: (),
	'StoredModules': lambda N, E: (),
	'HeatWarning': lambda N, E: (),
	'CodexEntry': lambda N, E: (),
	'RepairAll': lambda N, E: (),
	'Shipyard': lambda N, E: (),
	'StoredShips': lambda N, E: (),
	'ShipyardBuy': lambda N, E: (),
	'ShipyardNew': lambda N, E: (),
	'EscapeInterdiction': lambda N, E: (),
	'Repair': lambda N, E: (),
	'ModuleBuy': lambda N, E: (),
	'ModuleStore': lambda N, E: (),
	'BuyAmmo': lambda N, E: (),
	'UnderAttack': lambda N, E: (),
	'Bounty': lambda N, E: (),
	'HullDamage': lambda N, E: (),
	'Interdicted': lambda N, E: (),
	'Died': lambda N, E: (),
	'Resurrect': lambda N, E: (),
	'ModuleSwap': lambda N, E: (),
	'ModuleRetrieve': lambda N, E: (),
	'BuyTradeData': lambda N, E: (),
	'ModuleSellRemote': lambda N, E: (),
	'BuyExplorationData': lambda N, E: (),
	'MultiSellExplorationData': lambda N, E: (),
	'FuelScoop': lambda N, E: (),
	'FSSAllBodiesFound': lambda N, E: (),
	'EngineerContribution': lambda N, E: (),
	'CommitCrime': lambda N, E: (),
	'CollectCargo': lambda N, E: (),
	'FSSDiscoveryScan': lambda N, E: (),
	'MarketBuy': lambda N, E: (),
	'EngineerCraft': lambda N, E: (),
	'SAAScanComplete': lambda N, E: (),
	'SAASignalsFound': lambda N, E: (),
	'FetchRemoteModule': lambda N, E: (),
	'ShipyardSwap': lambda N, E: (),
	'ShipyardTransfer': lambda N, E: (),
	'SellExplorationData': lambda N, E: (),
	'Touchdown': lambda N, E: (),
	'Liftoff': lambda N, E: (),
	'LaunchSRV': lambda N, E: (),
	'DockSRV': lambda N, E: (),
	'MarketSell': lambda N, E: (),
	'PayBounties': lambda N, E: (),
	'EjectCargo': lambda N, E: (),
	'PayFines': lambda N, E: (),
	'Synthesis': lambda N, E: (),
	'ModuleSell': lambda N, E: (),
	'Passengers': lambda N, E: (),
	'DataScanned': lambda N, E: (),
	'Screenshot': lambda N, E: (),
	'MaterialTrade': lambda N, E: (),
	'SetUserShipName': lambda N, E: (),
	'BuyDrones': lambda N, E: (),
	'SellDrones': lambda N, E: (),
	'TechnologyBroker': lambda N, E: (),
	'SearchAndRescue': lambda N, E: (),
	'MiningRefined': lambda N, E: (),
	'LaunchDrone': lambda N, E: (),
	'ProspectedAsteroid': lambda N, E: (),
	#'Missions': lambda N, E: (),
	#'MissionAbandoned': lambda N, E: (),
	#'MissionAccepted': lambda N, E: (),
	#'MissionCompleted': lambda N, E: (),
	#'MissionRedirected': lambda N, E: (),
    # Handle events ---------------------------------------------------
    "Commander": lambda N,E: None if journal_story["Commanders"].update({
        E.get('FID'):{k:E.get(k,set()) for k in ['FID', 'Name', 'Missions']}
    }) else journal_story.update({"current":journal_story["Commanders"].get(E.get('FID'))}),
        
    "Materials": lambda N,E: journal_story["current"].update({N: {E.get("timestamp"): {
       k:{o.get("Name"):o.get("Count") for o in E.get(k)} for k in ["Raw","Manufactured","Encoded"]
    } }}),
    "Rank": lambda N,E:journal_story["current"].update({N:{E.pop("timestamp"):E}}),
    "Progress": lambda N,E:journal_story["current"].update({N:{E.pop("timestamp"):E}}),
    "Reputation": lambda N,E:journal_story["current"].update({N:{E.pop("timestamp"):E}}),
	#"Missions": lambda N,E:journal_story.get('current').get('Missions').update([M['MissionID'] for M in E['Active']]),
	#'MissionAccepted': lambda N, E: missions.update({E['MissionID']:{
    #    k:E.get(k) for k in ['LocalisedName', 'Expiry', 'DestinationSystem', 'DestinationStation']
    #}}),
    

}

In [ ]:
from edcompanion import events
importlib.reload(events)
entrytime = 0
jumptimes = []
body_id = 0
system_name = ''
rankings = dict(
    Progress = dict(),
    Rank = dict()
    
)
modules = []
ships = {}
my_materials = {}
event_seqnr = 0
unknown_events = {}
starpos = None

for event in events.edc_track_journal(logpath, backlog=6):
    timestamp = make_datetime(event.get("timestamp"))
    eventname = event.pop("event")
    
    if not jumptimes:
        entrytime=timestamp.timestamp()
        jumptimes.append(timestamp.timestamp())
    if not system_name and eventname == 'Location':
        system_name = event.get('StarSystem')
    starpos = np.asarray(event.get('StarPos', starpos))
        
    sys.stdout.write(
        f"\r{str(timestamp)[:-6]:20} {event_seqnr:5} "+
        f"{eventname:14} {timestamp.timestamp()-jumptimes[-1]:9,.0F} | {system_name:26} | ")
    
    if eventname not in journal_events and eventname not in unknown_events:
        sys.stdout.write(f"{event_seqnr:5} \n{json.dumps(event, indent=4, sort_keys=False)}")
        unknown_events[eventname]=event
        #break
    if eventname in journal_events:
        journal_events.get(eventname)(eventname, event)
    event_seqnr += 1
        
print(f"\nDone")
#prettyprint(journal_story['current']['Missions'])
#print(maxlen)

In [ ]:
[x for x in journal_story['current']['Missions']]

In [ ]:
missions

In [ ]:
for s in [f"\t'{str(k)}': lambda N, E: ()," for k in unknown_events]:
    print(s)

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)

# Companion API

In [ ]:
apiparams = dict(
    base_url="http://127.0.0.1:8000",
    headers=dict(
        authorization='9VknM2Gq2UcyM9tXJm43k7M4zVDNhVcQ' #os.getenv("USR_TOKEN")
    )
)
print(apiparams)

In [ ]:
readparams=dict(
    realm="edsm",
    # name="Ix",
    filter=json.dumps(dict(
        include=dict(stations='inner'),
        where=dict(name='Ix')
    )),
    format="json/split-index"
)

jsondata = None
t0=0
async with aiohttp.ClientSession(**apiparams) as session:
    t0 = time.perf_counter_ns()
    req = await session.get("/systems", params=readparams)
    t1 = time.perf_counter_ns()
    #print(req.headers)   
    #print(req.headers)  
    jsondata= await req.json()
    if jsondata:
        df = pd.read_json((await req.read()).decode("utf-8"), orient='split')
    #fulldata = np.asarray(await req.json())
    #print(f"Query took {(t1-t0) / 1e6} ms for {fulldata.shape[0]} rows")
    print(f"Query took {round((t1-t0) / 1e6,2)} ms for {len(df)} rows")

    print(req.url)



In [ ]:
edsy_export = json.loads('''
[{"header":{"appName":"EDSY","appVersion":308119901,"appURL":"https://edsy.org/#/L=GB00000H4C0SC0,,,9p300A5UG054_W0AN8G02r_W0AdsI05L_W0AsOG02m_W0B60J035_W0BOwI030_W0Bcg00,,5223034a300Nc90mpT7Q4G03N_W0mpU6e4B02jw00"},"data":{"event":"Loadout","Ship":"krait_light","ShipName":"","ShipIdent":"","HullValue":35732880,"ModulesValue":49907240,"UnladenMass":325.2,"CargoCapacity":0,"MaxJumpRange":68.315657,"FuelCapacity":{"Main":32,"Reserve":0.63},"Rebuy":4282006,"Modules":[{"Slot":"CargoHatch","Item":"modularcargobaydoor","On":false,"Priority":4},{"Slot":"Armour","Item":"krait_light_armour_grade1","On":true,"Priority":0,"Value":0},{"Slot":"PowerPlant","Item":"int_powerplant_size3_class5","On":true,"Priority":0,"Value":480410,"Engineering":{"BlueprintName":"PowerPlant_Stealth","Level":4,"Quality":1,"ExperimentalEffect":"special_powerplant_cooled","Modifiers":[{"Label":"Mass","Value":2.9,"OriginalValue":2.5},{"Label":"PowerCapacity","Value":10.56,"OriginalValue":12},{"Label":"HeatEfficiency","Value":0.162,"OriginalValue":0.4}]}},{"Slot":"MainEngines","Item":"int_engine_size4_class2","On":true,"Priority":0,"Value":59630,"Engineering":{"BlueprintName":"Engine_Tuned","Level":3,"Quality":1,"ExperimentalEffect":"special_engine_cooled","Modifiers":[{"Label":"Mass","Value":4.2,"OriginalValue":4},{"Label":"Integrity","Value":58.88,"OriginalValue":64},{"Label":"PowerDraw","Value":3.9852,"OriginalValue":3.69},{"Label":"EngineOptimalMass","Value":296.1,"OriginalValue":315},{"Label":"EngineOptPerformance","Value":118,"OriginalValue":100},{"Label":"EngineHeatRate","Value":0.702,"OriginalValue":1.3}]}},{"Slot":"FrameShiftDrive","Item":"int_hyperdrive_size5_class5","On":true,"Priority":2,"Value":5103950,"Engineering":{"BlueprintName":"FSD_LongRange","Level":5,"Quality":1,"ExperimentalEffect":"special_fsd_cooled","Modifiers":[{"Label":"Mass","Value":26,"OriginalValue":20},{"Label":"Integrity","Value":102,"OriginalValue":120},{"Label":"PowerDraw","Value":0.69,"OriginalValue":0.6},{"Label":"FSDOptimalMass","Value":1627.5,"OriginalValue":1050},{"Label":"FSDHeatRate","Value":24.3,"OriginalValue":27}]}},{"Slot":"LifeSupport","Item":"int_lifesupport_size4_class2","On":true,"Priority":0,"Value":28370,"Engineering":{"BlueprintName":"Misc_LightWeight","Level":3,"Quality":1,"Modifiers":[{"Label":"Mass","Value":1.4,"OriginalValue":4},{"Label":"Integrity","Value":50.4,"OriginalValue":72}]}},{"Slot":"PowerDistributor","Item":"int_powerdistributor_size4_class2","On":true,"Priority":3,"Value":28370,"Engineering":{"BlueprintName":"PowerDistributor_HighFrequency","Level":4,"Quality":1,"ExperimentalEffect":"special_powerdistributor_fast","Modifiers":[{"Label":"WeaponsCapacity","Value":22.1184,"OriginalValue":24},{"Label":"WeaponsRecharge","Value":3.67744,"OriginalValue":2.6},{"Label":"EnginesCapacity","Value":15.6672,"OriginalValue":17},{"Label":"EnginesRecharge","Value":1.98016,"OriginalValue":1.4},{"Label":"SystemsCapacity","Value":15.6672,"OriginalValue":17},{"Label":"SystemsRecharge","Value":1.98016,"OriginalValue":1.4}]}},{"Slot":"Radar","Item":"int_sensors_size6_class1","On":true,"Priority":2,"Value":88980,"Engineering":{"BlueprintName":"Sensor_LightWeight","Level":4,"Quality":1,"Modifiers":[{"Label":"Mass","Value":14,"OriginalValue":40},{"Label":"Integrity","Value":61.2,"OriginalValue":102},{"Label":"SensorTargetScanAngle","Value":24,"OriginalValue":30}]}},{"Slot":"FuelTank","Item":"int_fueltank_size5_class3","On":true,"Priority":0,"Value":97750},{"Slot":"Slot01_Size6","Item":"int_fuelscoop_size6_class5","On":true,"Priority":3,"Value":28763610},{"Slot":"Slot02_Size5","Item":"int_guardianfsdbooster_size5","On":true,"Priority":3,"Value":6483100},{"Slot":"Slot03_Size5","Item":"int_repairer_size5_class5","On":false,"Priority":1,"Value":8503060},{"Slot":"Slot05_Size3","Item":"int_shieldgenerator_size3_class2","On":true,"Priority":0,"Value":18810,"Engineering":{"BlueprintName":"ShieldGenerator_Optimised","Level":5,"Quality":1,"ExperimentalEffect":"special_shield_lightweight","Modifiers":[{"Label":"Mass","Value":0.9,"OriginalValue":2},{"Label":"Integrity","Value":28.5,"OriginalValue":38},{"Label":"PowerDraw","Value":0.864,"OriginalValue":1.44},{"Label":"ShieldGenOptimalMass","Value":155.1,"OriginalValue":165},{"Label":"ShieldGenStrength","Value":103.5,"OriginalValue":90}]}},{"Slot":"Slot08_Size2","Item":"int_dronecontrol_repair_size1_class2","On":false,"Priority":3,"Value":1200},{"Slot":"Slot09_Size1","Item":"int_detailedsurfacescanner_tiny","On":true,"Priority":0,"Value":250000}]}}]
''')

In [ ]:
prettyprint(edsy_export)

In [ ]:
jsondata

# Database

In [ ]:
system_info_lhs2661 = get_edsm_info("LHS 2661")
print(system_info_lhs2661)

In [ ]:
[B for B in system_info_lhs2661.get('bodies') if ' 6 ' in B.get('name')]

## Creation

### Systems

In [ ]:
# Drop te existing sytems table

assert False, "Swap the comments on these two lines if you really want to recreate the systems table"
#await pgpool.execute(f"DROP TABLE edsm.systems;")
await pgpool.execute(f"""
    CREATE TABLE IF NOT EXISTS edsm.systems (
        system_id BIGINT NOT NULL,
        x         REAL NOT NULL,
        y         REAL NOT NULL,
        z         REAL NOT NULL
        name      TEXT NOT NULL,
        bodycount INT DEFAULT 0
    );
    DROP INDEX IF EXISTS systems_name_unique
    DROP INDEX IF EXISTS systems_name_idx
    DROP INDEX IF EXISTS systems_id_unique
    DROP INDEX IF EXISTS systems_id_idx
    CREATE UNIQUE INDEX IF NOT EXISTS systems_id_unique ON edsm.systems (system_id);
    CREATE INDEX IF NOT EXISTS systems_name_idx ON edsm.systems (name);

    CREATE INDEX IF NOT EXISTS systems_x_idx ON edsm.systems (x);
    CREATE INDEX IF NOT EXISTS systems_y_idx ON edsm.systems (y);
    CREATE INDEX IF NOT EXISTS systems_z_idx ON edsm.systems (z); 

""")

systems_insert_query = f"""
    INSERT INTO edsm.systems (system_id, name, x, y, z, bodycount) 
    VALUES ($1, $2, $3, $4, $5, $6) 
    ON CONFLICT (system_id) DO UPDATE SET 
        name = $2,
        x=$3, y=$4, z=$5,
        bodycount = $6

"""


#### Alter existing systems table

In [ ]:
assert False, "Comment this assert if you really want to alter the table"
await pgpool.execute(f"""
        ALTER TABLE edsm.systems ADD COLUMN system_id BIGINT;
        ALTER TABLE edsm.systems ADD COLUMN bodycount INT DEFAULT 0;
        CREATE INDEX IF NOT EXISTS systems_id_idx ON edsm.systems (system_id);

""")

In [ ]:
systems_insert_query = f"""
    INSERT INTO edsm.systems (system_id, name, x, y, z, bodycount) 
    VALUES ($1, $2, $3, $4, $5, $6) 
    ON CONFLICT (name) DO UPDATE SET 
        system_id = $1,
        x=$3, y=$4, z=$5,
        bodycount = $6

"""

In [ ]:
await pgpool.execute("""
    DELETE FROM edsm.systems a
    WHERE   a.ctid <> (SELECT min(b.ctid)
                     FROM   edsm.systems b
                     WHERE  a.name = b.name );"""
)

In [ ]:
await pgpool.execute("""
    DELETE FROM edsm.systems a
    WHERE   a.ctid <> (SELECT min(b.ctid)
                     FROM   edsm.systems b
                     WHERE  a.system_id = b.system_id );"""
)

In [ ]:
await pgpool.execute(f"""
    DROP INDEX IF EXISTS systems_name_unique
    DROP INDEX IF EXISTS systems_name_idx
    DROP INDEX IF EXISTS systems_id_unique
    DROP INDEX IF EXISTS systems_id_idx
    CREATE UNIQUE INDEX IF NOT EXISTS systems_id_unique ON edsm.systems (system_id);
    CREATE INDEX IF NOT EXISTS systems_name_idx ON edsm.systems (name);

    CREATE INDEX IF NOT EXISTS systems_x_idx ON edsm.systems (x);
    CREATE INDEX IF NOT EXISTS systems_y_idx ON edsm.systems (y);
    CREATE INDEX IF NOT EXISTS systems_z_idx ON edsm.systems (z); 

""")

systems_insert_query = f"""
    INSERT INTO edsm.systems (system_id, name, x, y, z, bodycount) 
    VALUES ($1, $2, $3, $4, $5, $6) 
    ON CONFLICT (system_id) DO UPDATE SET 
        name = $2,
        x=$3, y=$4, z=$5,
        bodycount = $6

"""


### Populated

In [ ]:
# Drop te existing table:
assert False, "Swap the comments on these two lines if you really want to recreate the table"
#await pgpool.execute(f"DROP TABLE edsm.populated;")
await pgpool.execute(f"""
    CREATE TABLE IF NOT EXISTS edsm.populated (
      system_id      BIGINT NOT NULL
      population     REAL DEFAULT 0,
      faction        TEXT,
      allegiance     TEXT,
      government     TEXT,
      economy        TEXT,
      security       TEXT
    ) ;
    CREATE UNIQUE INDEX IF NOT EXISTS populated_system_id_unique  ON edsm.populated (system_id)
    CREATE INDEX IF NOT EXISTS populated_faction_idx ON edsm.populated (faction);
""")
populated_insert_query = f"""
    INSERT INTO edsm.populated (
        system_id,
        population, allegiance, government, 
        economy, security, faction
    ) 
    VALUES ($1, $2, $3, $4, $5, $6, $7) 
    ON CONFLICT (system_id) DO UPDATE SET 
        population=$2, allegiance=$3, government=$4, 
        economy=$5, security=$6, faction=$7

"""


### Bodies

In [ ]:
# Drop te existing sytems table:
assert False, "Swap the comments on these two lines if you really want to recreate the table"
#await pgpool.execute(f"DROP TABLE edsm.bodies;")
await pgpool.execute(f"""
    CREATE TABLE IF NOT EXISTS edsm.bodies (
        body_id   BIGINT NOT NULL
        system_id BIGINT NOT NULL,
        name      TEXT NOT NULL
    );
    CREATE UNIQUE INDEX IF NOT EXISTS bodies_id_unique ON edsm.bodies (body_id);
    CREATE INDEX IF NOT EXISTS bodies_system_id_idx ON edsm.bodies (system_id);

""")


#### stars

In [ ]:
# Drop te existing sytems table:
assert False, "Swap the comments on these three lines if you really want to recreate the table"
#await pgpool.execute(f"DROP TABLE edsm.stars;")
#await pgpool.execute(f"DROP TABLE edsm.mainstars;")
await pgpool.execute(f"""
    CREATE TABLE IF NOT EXISTS edsm.stars (
      subtype         TEXT NOT NULL,
      distance        REAL NOT NULL,
      solarmasses     REAL NOT NULL,
      temperature     REAL NOT NULL,
      magnitude       REAL NOT NULL,
      spectral_class  TEXT NOT NULL,
      luminosity      TEXT NOT NULL
    ) INHERITS (edsm.bodies);
""")
await pgpool.execute(f"""
    CREATE TABLE IF NOT EXISTS edsm.mainstars (
    ) INHERITS (edsm.stars);
""")
stars_insert_query = f"""
    INSERT INTO edsm.stars (
        body_id, system_id, distance, name, subtype,
        solarmasses, temperature, magnitude, spectral_class, luminosity
    ) 
    VALUES ($1, $2, $3, $4, $5, $6,$7, $8, $9, $10) 
    ON CONFLICT (body_id) DO UPDATE SET 
        system_id = $2,
        distance =$3, name=$4, subtype=$5,
        solarmasses=$6, temperature=$7, magnitude=$8, 
        spectral_class=$9, luminosity=$10

"""

In [ ]:
# Drop te existing sytems table:
assert False, "Swap the comments on these two lines if you really want to recreate the table"
#await pgpool.execute(f"DROP TABLE edsm.barycentres;")
await pgpool.execute(f"""
    CREATE TABLE IF NOT EXISTS edsm.barycentres (
    ) INHERITS (edsm.bodies);
""")
barycentres_insert_query = f"""
    INSERT INTO edsm.barycentres (
        body_id, system_id, name
        
    ) 
    VALUES ($1, $2, $3) 
    ON CONFLICT (body_id) DO UPDATE SET 
        system_id = $2, name=$3

"""

#### planets

In [ ]:
# Drop te existing table:
assert False, "Swap the comments on these two lines if you really want to recreate the table"
#await pgpool.execute(f"DROP TABLE edsm.planets;")
await pgpool.execute(f"""
    CREATE TABLE IF NOT EXISTS edsm.planets (
      subtype        TEXT NOT NULL,
      earthmasses    REAL NOT NULL,
      gravity        REAL NOT NULL,
      temperature    REAL NOT NULL
      landable       BOOL DEFAULT FALSE,
      volcanism      BOOL DEAFULT FALSE
      
    ) INHERITS (edsm.bodies);
""")
planets_insert_query = f"""
    INSERT INTO edsm.planets (
        body_id, system_id, distance, name, subtype,
        earthmasses, temperature, gravity, landable, volcanism
    ) 
    VALUES ($1, $2, $3, $4, $5, $6,$7, $8, $9, $10) 
    ON CONFLICT (body_id) DO UPDATE SET 
        system_id = $2,
        distance =$3, name=$4, subtype=$5,
        earthmasses=$6, temperature=$7, gravity=$8, 
        landable=$9, volcanism=$10

"""

### Stations

In [ ]:
# Drop te existing table:
assert False, "Swap the comments on these two lines if you really want to recreate the table"
#await pgpool.execute(f"DROP TABLE edsm.stations;")
await pgpool.execute(f"""
    CREATE TABLE IF NOT EXISTS edsm.stations (
        station_id   BIGINT NOT NULL,
        system_id    BIGINT NOT NULL,
        distance     REAL NOT NULL,
        name         TEXT NOT NULL,
        type         TEXT NOT NULL,
        faction      TEXT
      
    ) ;
    
    CREATE INDEX IF NOT EXISTS station_name_idx ON edsm.stations (name);
    CREATE INDEX IF NOT EXISTS station_faction_idx ON edsm.stations (faction);
    CREATE INDEX IF NOT EXISTS station_system_id_idx ON edsm.stations (system_id);

""")
stations_insert_query = f"""
    INSERT INTO edsm.stations (
        station_id, system_id,
        distance, name, type, faction
    )
    VALUES ($1, $2, $3, $4, $5, $6) 
    ON CONFLICT (station_id) DO UPDATE SET 
        system_id=$2, distance=$3, 
        name=$4, type=$5, faction=$6
    
"""

### Factions

In [ ]:
# Drop te existing table:
assert False, "Swap the comments on these two lines if you really want to recreate the table"
#await pgpool.execute(f"DROP TABLE edsm.factions;")
await pgpool.execute(f"""
    CREATE TABLE IF NOT EXISTS edsm.factions (
        system_id      BIGINT NOT NULL,
        faction        TEXT,
        allegiance     TEXT,
        government     TEXT,
        influence      REAL,
        state          TEXT
      
    ) ;
    
    CREATE UNIQUE INDEX IF NOT EXISTS factions_ass_faction_system on edsm.factions (faction, system_id)
    CREATE INDEX IF NOT EXISTS factions_system_id ON edsm.factions (system_id);
    CREATE INDEX IF NOT EXISTS factions_faction_idx ON edsm.factions (faction);

""")

In [ ]:
factions_insert_query = f"""
    INSERT INTO edsm.factions (
        system_id, faction,
        allegiance, government, 
        influence, state
    )
    VALUES ($1, $2, $3, $4, $5, $6) 
    ON CONFLICT (faction, system_id) DO UPDATE SET 
        allegiance=$3, government=$4, 
        influence=$5, state=$6
    
"""

## Insert

## Indexes

In [ ]:
await pgpool.execute(f"""

CREATE INDEX IF NOT EXISTS powers_system_id_idx ON edsm.powers (system_id);
CREATE INDEX IF NOT EXISTS powers_power_ixd ON edsm.powers (power); 


CREATE INDEX IF NOT EXISTS populated_system_id_idx ON edsm.populated (system_id);

""")


In [ ]:
await pgpool.execute("""
    DELETE FROM edsm.powers a
    WHERE   a.ctid <> (SELECT min(b.ctid)
                     FROM   edsm.powers b
                     WHERE  a.power = b.power AND a.system_id = b.system_id);"""
)

In [ ]:
await pgpool.execute(f"""
CREATE unique INDEX IF NOT EXISTS systems_id_uniq ON edsm.systems (system_id);
CREATE unique INDEX IF NOT EXISTS powers_powersystem_uniq ON edsm.powers (power, system_id);

""")


## Update